# **Intalling Necessary Libraries**

In [ ]:
!pip install llama-index
!pip install llama-index-llms-anthropic
!pip install llama-index-embeddings-huggingface

In [35]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()


from IPython.display import display, HTML

In [ ]:
ANTHROPIC_API_KEY="YOUR KEY"

In [40]:
from llama_index.llms.anthropic import Anthropic
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [41]:
from llama_index.core.agent import ReActAgent
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

**Set LLM and Embedding model**

In [42]:
llm = Anthropic(temperature=0.0, model='claude-3-opus-20240229',api_key=ANTHROPIC_API_KEY)

In [43]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

In [44]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

# **ReAct Agent over QueryEngine Tools**

In [45]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# **Load Data**

In [46]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

google_docs = SimpleDirectoryReader(input_files=["/content/data/goog-10-k-2023.pdf"]).load_data()
netflix_docs = SimpleDirectoryReader(input_files=["/content/data/Netflix-10-K-01262024.pdf"]).load_data()

# **Build Index**

In [47]:
google_index = VectorStoreIndex.from_documents(google_docs)
netflix_index = VectorStoreIndex.from_documents(netflix_docs)

# **Create QueryEngines**

In [48]:
google_engine = google_index.as_query_engine(similarity_top_k=3)
netflix__engine = netflix_index.as_query_engine(similarity_top_k=3)

# **Create QueryEngine Tools**

In [49]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=google_engine,
        metadata=ToolMetadata(
            name="google_10k",
            description=(
                "Provides information about google financials for year 2023. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=netflix__engine,
        metadata=ToolMetadata(
            name="netflix_10k",
            description=(
                "Provides information about Netflix financials for year 2023. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

# **ReAct Agent**

In [50]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
)

**Querying with ReAct Agent**

In [51]:
response = agent.chat("What was Google's revenue growth in 2023?")

HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: google_10k
Action Input: {'input': "What was Google's revenue growth in 2023?"}
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Observation: According to the context provided, Google's total revenues increased from $282.8 billion in 2022 to $307.4 billion in 2023, representing revenue growth of $24.6 billion.

The key drivers of this revenue growth in 2023 were:

- Google Search & other revenues increased by $12.6 billion
- YouTube ads revenues grew by $2.3 billion  
- Google subscriptions, platforms, and devices revenues increased by $5.6 billion, largely driven by growth in YouTube subscriptions and higher sales of Pixel devices
- Google Cloud revenues increased by $6.8 billion, primarily driven by growth in Google Cloud Platform and Google Workspace

However, Google Ne

In [52]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))

In [53]:
response = agent.chat(
    "Compare and contrast the revenue growth of Netflix and Google in 2023, then"
    " give an analysis"
)

HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Thought: The current language of the user is: English. I need to use tools to help me compare and contrast the revenue growth of Netflix and Google in 2023 and provide an analysis.
Action: google_10k
Action Input: {'input': "What was Google's revenue growth in 2023 in dollar amount and percentage?"}
HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
Observation: Based on the financial information provided in the context, Google's total revenues increased from $282,836 million in 2022 to $307,394 million in 2023. This represents a revenue growth of $24,558 million in dollar amount.

To calculate the percentage growth:
Percent change = (2023 revenue - 2022 revenue) / 2022 revenue x 100
            = (307,394 - 282,836) / 282,836 x 100
            = 24,558 / 282,836 x 100
            = 8.68%

Therefore, Google's revenue grew by $24,558 million or 8.68% in 2023 compared to 2022.
HTTP Request:

In [54]:
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))